<a href="https://colab.research.google.com/github/mellow-75/hackathon-/blob/main/Taxonomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('/content/drive/MyDrive/hackathon/Doceree-HCP_Train.csv',encoding = "ISO-8859-1")
df.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0


In [4]:
df.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               113937 non-null  int64  
 1   DEVICETYPE       113937 non-null  object 
 2   PLATFORM_ID      113937 non-null  int64  
 3   BIDREQUESTIP     113937 non-null  object 
 4   USERPLATFORMUID  113933 non-null  object 
 5   USERCITY         107578 non-null  object 
 6   USERZIPCODE      109345 non-null  float64
 7   USERAGENT        113935 non-null  object 
 8   PLATFORMTYPE     113937 non-null  object 
 9   CHANNELTYPE      113937 non-null  object 
 10  URL              113937 non-null  object 
 11  KEYWORDS         113937 non-null  object 
 12  TAXONOMY         32313 non-null   object 
 13  IS_HCP           113936 non-null  float64
dtypes: float64(2), int64(2), object(10)
memory usage: 12.2+ MB


In [6]:
ohe = pd.get_dummies(df,columns = ['DEVICETYPE','PLATFORM_ID','PLATFORMTYPE'])
ohe.head()

,ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,...,PLATFORM_ID_12,PLATFORM_ID_13,PLATFORM_ID_14,PLATFORM_ID_15,PLATFORM_ID_16,PLATFORMTYPE_Electronic Health Record,PLATFORMTYPE_Medical News Platform,PLATFORMTYPE_Online Learning Portal,PLATFORMTYPE_Online Medical Journal,PLATFORMTYPE_Tele-medicine Platform
0,1001,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,...,0,0,0,0,0,0,0,0,1,0
1,1002,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,...,0,0,0,0,0,0,0,0,1,0
2,1003,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,...,0,0,0,0,0,0,0,0,1,0
3,1004,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,...,0,0,0,0,0,0,0,0,1,0
4,1005,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,...,0,0,0,0,0,0,0,0,1,0


In [7]:
drop = ohe.drop(['USERPLATFORMUID','ID','USERCITY','USERAGENT','CHANNELTYPE','URL'],axis = 1)
drop.head()

,BIDREQUESTIP,USERZIPCODE,KEYWORDS,TAXONOMY,IS_HCP,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,...,PLATFORM_ID_12,PLATFORM_ID_13,PLATFORM_ID_14,PLATFORM_ID_15,PLATFORM_ID_16,PLATFORMTYPE_Electronic Health Record,PLATFORMTYPE_Medical News Platform,PLATFORMTYPE_Online Learning Portal,PLATFORMTYPE_Online Medical Journal,PLATFORMTYPE_Tele-medicine Platform
0,170.173.0.22,97206.0,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,65.216.253.25,22202.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,66.232.79.22,83654.0,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,137.54.125.246,229114624.0,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,174.202.231.99,77008.0,Critical Care|Emergency Medicine|General Pract...,NaN,0.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split('|'))

In [9]:
bid = tfidf.fit_transform(df['BIDREQUESTIP'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
bid = pd.DataFrame(bid.toarray(),columns = tfidf.get_feature_names_out())
bid.head()

,100.0.107.20,100.0.110.97,100.0.189.223,100.0.195.112,100.0.197.5,100.0.35.76,100.0.54.56,100.0.64.3,100.0.73.120,100.0.8.23,...,99.98.181.26,99.98.210.91,99.98.249.51,99.98.60.121,99.98.79.185,99.98.80.135,99.99.111.75,99.99.141.77,99.99.180.104,99.99.185.108
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
bid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Columns: 33664 entries, 100.0.107.20 to 99.99.185.108
dtypes: float64(33664)
memory usage: 28.6 GB


In [12]:
drop = drop.drop(['BIDREQUESTIP'],axis = 1)

In [13]:
drop.head()

,USERZIPCODE,KEYWORDS,TAXONOMY,IS_HCP,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,...,PLATFORM_ID_12,PLATFORM_ID_13,PLATFORM_ID_14,PLATFORM_ID_15,PLATFORM_ID_16,PLATFORMTYPE_Electronic Health Record,PLATFORMTYPE_Medical News Platform,PLATFORMTYPE_Online Learning Portal,PLATFORMTYPE_Online Medical Journal,PLATFORMTYPE_Tele-medicine Platform
0,97206.0,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,22202.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,83654.0,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,229114624.0,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,77008.0,Critical Care|Emergency Medicine|General Pract...,NaN,0.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
kywrds = tfidf.fit_transform(df['KEYWORDS'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
kywrds = pd.DataFrame(kywrds.toarray(),columns = tfidf.get_feature_names_out())

In [17]:
kywrds.head()

,,abatacept,ablation techniques,abortion,abscess,acanthamoeba keratitis,acetaminophen,acid-base imbalance,aco,acos,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
kywrds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Columns: 1216 entries,  to zoster sine herpete
dtypes: float64(1216)
memory usage: 1.0 GB


In [19]:
fa = pd.concat([drop,kywrds],axis = 1)
fa.head()

,USERZIPCODE,KEYWORDS,TAXONOMY,IS_HCP,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,97206.0,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22202.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,83654.0,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,229114624.0,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,77008.0,Critical Care|Emergency Medicine|General Pract...,NaN,0.0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
fa = fa.drop('KEYWORDS',axis = 1)
fa.head()

,USERZIPCODE,TAXONOMY,IS_HCP,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,PLATFORM_ID_3,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,97206.0,NaN,0.0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22202.0,NaN,0.0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,83654.0,NaN,0.0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,229114624.0,2084P0800X,1.0,1,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,77008.0,NaN,0.0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
fa.drop('IS_HCP',axis = 1,inplace = True)

In [23]:
fa.head()

,USERZIPCODE,TAXONOMY,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,PLATFORM_ID_3,PLATFORM_ID_4,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,97206.0,NaN,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22202.0,NaN,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,83654.0,NaN,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,229114624.0,2084P0800X,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,77008.0,NaN,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
fa = fa.dropna(how = 'any',axis = 0)
null = dict(fa.isnull().sum())

In [25]:
nl_ks = null.keys()
for key in nl_ks:
  if(null[key]>0):
    print(0)

since no output so there are no null values

In [28]:
fa.info() #object is taxonomy

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32313 entries, 3 to 113936
Columns: 1242 entries, USERZIPCODE to zoster sine herpete
dtypes: float64(1217), object(1), uint8(24)
memory usage: 301.3+ MB


In [29]:
y_df = fa['TAXONOMY']
y_df.head()

3     2084P0800X
8     2084P0804X
21    2084N0400X
29    2084V0102X
35    207RC0000X
Name: TAXONOMY, dtype: object

In [31]:
y_df = pd.get_dummies(y_df,columns = ['TAXONOMY'])
y_df.head()

,103T00000X,103TC0700X,1041C0700X,106H00000X,111N00000X,122300000X,1223P0221X,146D00000X,152W00000X,163W00000X,...,364S00000X,364SF0001X,364SP0807X,364SP0808X,364SP0809X,367500000X,367A00000X,367H00000X,376K00000X,390200000X
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
print(fa.shape,y_df.shape)

(32313, 1242) (32313, 207)


In [35]:
null = dict(y_df.isnull().sum())
nl_ks = null.keys()
for key in nl_ks:
  if(null[key]>0):
    print(0)

no null in y values as well so we are ready to train

In [37]:
X = fa.drop(['TAXONOMY'],axis = 1)
X.head()

,USERZIPCODE,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,PLATFORM_ID_3,PLATFORM_ID_4,PLATFORM_ID_5,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
3,229114624.0,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,91344.0,0,1,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,912031130.0,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,90620.0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,2143.0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
sc = StandardScaler()

In [40]:
X_scaled = sc.fit_transform(X)

In [56]:
classes = list(y_df.columns)

In [57]:
classes[0]

'103T00000X'

In [41]:
X.shape

(32313, 1241)

In [45]:
y = y_df.to_numpy()
y.shape

(32313, 207)

In [46]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.4)

In [48]:
rf = RandomForestClassifier()

In [49]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [50]:
y_pred_val = rf.predict(X_val)

In [51]:
y_pred_val

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [59]:
m = y_val.shape[0]
n = y_val.shape[1]
y_true = np.zeros([m,1])
y_pred = np.zeros([m,1])
for i in range(m):
  for j in range(n):
    if(y_pred_val[i][j] == 1):
      y_pred[i] = j
    if(y_val[i][j] == 1):
      y_true[i] = j

In [60]:
y_true

array([[112.],
       [143.],
       [145.],
       ...,
       [151.],
       [112.],
       [144.]])

In [61]:
y_pred

array([[112.],
       [143.],
       [145.],
       ...,
       [149.],
       [112.],
       [143.]])

In [65]:
accuracy_score(y_true,y_pred)

0.5632833049667337